In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import os

C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\brgoh\anaconda33\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
df = pd.read_csv('tweets_stocks_combined_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2219 non-null   float64
 1   cleaned_text            2219 non-null   object 
 2   favorites               2219 non-null   int64  
 3   retweets                2219 non-null   int64  
 4   date                    2219 non-null   object 
 5   tweet_datetime          2219 non-null   object 
 6   date_part               2219 non-null   object 
 7   time_part               2219 non-null   object 
 8   hour                    2219 non-null   int64  
 9   year                    2219 non-null   int64  
 10  month                   2219 non-null   int64  
 11  cleaned_text_2          2219 non-null   object 
 12  datetime_60mins_after   2219 non-null   object 
 13  price_60mins_after      2219 non-null   float64
 14  datetime_now            2219 non-null   

In [3]:
model_a_df = df.copy()

In [4]:
model_a_df

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,cleaned_text_2,datetime_60mins_after,price_60mins_after,datetime_now,price_now,60mins_price_diff_abs,60mins_price_diff_perc,prev_60mins_prices
0,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,thank you rand,2017-11-28 11:50:00,261.485000,2017-11-28 10:50:00,261.100000,0.385000,0.001475,"260.96,261.03,261.01,261.015,261.04,261.01,261..."
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,join me live from fort myer in arlington virgi...,2017-08-22 10:00:00,244.260000,2017-08-22 09:00:00,243.670000,0.590000,0.002421,"243.76,243.79,243.85,243.86,243.81,243.78,243...."
2,8.939700e+17,Thank you Nicole!,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,thank you nicole,2017-05-09 08:01:00,239.940000,2017-05-09 07:01:00,239.875000,0.065000,0.000271,"239.73,239.73,239.73,239.73,239.73,239.73,239...."
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,thank you to shawn steel for the nice words on,2017-03-08 05:44:00,237.022857,2017-03-08 04:44:00,236.880000,0.142857,0.000603,"236.84,236.84,236.84,236.85333333333332,236.86..."
4,8.787250e+17,MAKE AMERICA GREAT AGAIN!,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,make america great again,2017-06-25 06:23:00,243.326476,2017-06-25 05:23:00,243.320762,0.005714,0.000023,"243.31790476190474,243.31799999999998,243.3180..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,9.990960e+17,If the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,if the person placed very early into my campai...,2018-05-23 10:13:00,271.930000,2018-05-23 09:13:00,271.040000,0.890000,0.003284,"271.18,271.16,271.18,271.15,271.08,271.07,271...."
2215,9.874600e+17,So General Michael Flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,so general michael flynn’s life can be totally...,2018-04-20 19:34:00,267.025000,2018-04-20 18:34:00,266.820000,0.205000,0.000768,"266.74625000000003,266.7475,266.74875,266.75,2..."
2216,9.870960e+17,"My thoughts, prayers and condolences are with ...",62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,my thoughts prayers and condolences are with t...,2018-04-20 07:30:00,269.070000,2018-04-20 06:30:00,268.620000,0.450000,0.001675,"268.77,268.74333333333334,268.71666666666664,2..."
2217,9.863570e+17,Today’s Court decision means that Congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,today’s court decision means that congress mus...,2018-04-18 06:34:00,270.695000,2018-04-18 05:34:00,270.600000,0.095000,0.000351,"270.73,270.7,270.72749999999996,270.755,270.78..."


# Model A (only word vectors)

In [5]:
model_a_X = model_a_df.loc[:, 'cleaned_text_2']
model_a_y = model_a_df.loc[:, '60mins_price_diff_perc']*100

In [6]:
model_a_X_train, model_a_X_test, model_a_y_train, model_a_y_test = train_test_split(model_a_X, model_a_y, test_size=0.33, random_state=42)

In [7]:
model_a_corpus_list = []

for i in model_a_X_train:
    model_a_corpus_list.append(i.split())

In [8]:
model_a_word2vec_model = Word2Vec(model_a_corpus_list, min_count=1, size=100)
model_a_pretrained_weights = model_a_word2vec_model.wv.vectors

In [9]:
model_a_num_words = [len(i) for i in model_a_corpus_list]
model_a_longest_sentence_len = max(model_a_num_words)

In [10]:
def word2vec_sentence_to_indices_padded(sentences, longest_sentence_len, word2vec_model):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='pre')

In [11]:
def glove_sentence_to_indices_padded(sentences, longest_sentence_len, glove_model):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='pre')

In [12]:
model_a_X_train_padded = word2vec_sentence_to_indices_padded(model_a_X_train, model_a_longest_sentence_len, model_a_word2vec_model)
model_a_X_test_padded = word2vec_sentence_to_indices_padded(model_a_X_test, model_a_longest_sentence_len, model_a_word2vec_model)

In [ ]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(model_a_X_train)
sequences = tokenizer.texts_to_sequences(model_a_X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
w2c = dict()
for item in model_a_word2vec_model.wv.vocab:
    w2c[item]=model_a_word2vec_model.wv.vocab[item].count
w2c_sorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
print(w2c_sorted)

In [ ]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

In [ ]:
words_w2v = list(w2c.keys())
words_gloves = list(glove_word_count.keys())
extra = []
for word in words_w2v:
    if word not in words_gloves:
        extra.append(word)
print(extra)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
vocab_size_glove, embedding_size_glove = embedding_matrix.shape
print(vocab_size_glove)
print(embedding_size_glove)
print(count)
print(skipped_words)
embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=model_a_longest_sentence_len,
                            trainable=False)

In [13]:
def create_model_a_2seq_tanh(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
#     model.add(embedding_layer_glove)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='tanh'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [14]:
def create_model_a_1seq_tanh(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
#     model.add(embedding_layer_glove)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='tanh'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [15]:
def create_model_a_1seq_sigmoid(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
#     model.add(embedding_layer_glove)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [16]:
def create_model_a_2seq_sigmoid(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
#     model.add(embedding_layer_glove)
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

### 2 LSTMs, tanh activation

In [17]:
model_a_2seq_tanh = create_model_a_2seq_tanh(model_a_pretrained_weights, model_a_longest_sentence_len)
opt = keras.optimizers.Adam(learning_rate=0.001)
model_a_2seq_tanh.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
model_a_2seq_tanh.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/model_a_2seq_tanh {dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a_2seq_tanh.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

Train on 1188 samples, validate on 298 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.0298 - mean_absolute_error: 0.0927
Epoch 00001: val_loss improved from inf to 0.02292, saving model to ./model_a_checkpoint/model_a_2seq_tanh 11112020 1302h.h5
1188/1188 [==============================] - 14s 12ms/sample - loss: 0.0305 - mean_absolute_error: 0.0933 - val_loss: 0.0229 - val_mean_absolute_error: 0.0700
Epoch 2/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.0292 - mean_absolute_error: 0.0888
Epoch 00002: val_loss improved from 0.02292 to 0.02283, saving model to ./model_a_checkpoint/model_a_2seq_tanh 11112020 1302h.h5
1188/1188 [==============================] - 10s 8ms/sample - loss: 0.0291 - mean_absolute_error: 0.0885 - val_loss: 0.0228 - val_mean_absolute_error: 0.0699
Epoch 3/50
1184/1188 [============================>.] - ETA: 0s - loss

1184/1188 [============================>.] - ETA: 0s - loss: 0.0289 - mean_absolute_error: 0.0845
Epoch 00024: val_loss did not improve from 0.02282
1188/1188 [==============================] - 10s 9ms/sample - loss: 0.0288 - mean_absolute_error: 0.0844 - val_loss: 0.0228 - val_mean_absolute_error: 0.0692
Epoch 25/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.0288 - mean_absolute_error: 0.0838
Epoch 00025: val_loss did not improve from 0.02282
1188/1188 [==============================] - 11s 9ms/sample - loss: 0.0287 - mean_absolute_error: 0.0836 - val_loss: 0.0229 - val_mean_absolute_error: 0.0701
Epoch 26/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.0287 - mean_absolute_error: 0.0835
Epoch 00026: val_loss did not improve from 0.02282
1188/1188 [==============================] - 9s 8ms/sample - loss: 0.0287 - mean_absolute_error: 0.0835 - val_loss: 0.0228 - val_mean_absolute_error: 0.0690
Epoch 27/50
1184/1188 [============================>.] -

### 1 LSTM, tanh activation

In [ ]:
model_a_1seq_tanh = create_model_a_1seq_tanh(model_a_pretrained_weights, model_a_longest_sentence_len)
opt = keras.optimizers.Adam(learning_rate=0.001)
model_a_1seq_tanh.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
model_a_1seq_tanh.summary()

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/model_a_1seq_tanh {dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a_1seq_tanh.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

### 2 LSTMs, sigmoid activation

In [ ]:
model_a_2seq_sigmoid = create_model_a_2seq_sigmoid(model_a_pretrained_weights, model_a_longest_sentence_len)
opt = keras.optimizers.Adam(learning_rate=0.001)
model_a_2seq_sigmoid.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
model_a_2seq_sigmoid.summary()

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/model_a_2seq_sigmoid {dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a_2seq_sigmoid.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

### 1 LSTM, sigmoid activation

In [ ]:
model_a_1seq_sigmoid = create_model_a_1seq_sigmoid(model_a_pretrained_weights, model_a_longest_sentence_len)
opt = keras.optimizers.Adam(learning_rate=0.001)
model_a_1seq_sigmoid.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
model_a_1seq_sigmoid.summary()

In [ ]:
loaded_model = create_model_a(model_a_pretrained_weights, model_a_longest_sentence_len)
loaded_model.load_weights('./model_a_checkpoint/11112020 1242h (tanh).h5')
loaded_model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
y_pred = loaded_model.predict(model_a_X_test_padded)

In [ ]:
print(y_pred)

In [ ]:
from sklearn import metrics

model_a.evaluate(model_a_X_test_padded, model_a_y_test)

In [ ]:
# tanh

from sklearn import metrics
dev_loss, dev_acc = loaded_model.evaluate(model_a_X_test_padded,  model_a_y_test, verbose=1)


print( np.sqrt( metrics.mean_squared_error(model_a_y_train, loaded_model.predict(model_a_X_train_padded)) ) )
print( np.sqrt( metrics.mean_squared_error(model_a_y_test, loaded_model.predict(model_a_X_test_padded)) ) )
print( np.sqrt( metrics.mean_squared_error(model_a_y_test, 0*model_a_y_test ) ) )

In [ ]:
model_a_y_train

In [ ]:
min(model_a_y_train)

# Model B (word vectors + price history)

In [ ]:
model_b_df = df.dropna(subset=['cleaned_text', 'prev_60mins_prices'])

model_b_X = model_b_df.loc[:, ['cleaned_text_2', 'prev_60mins_prices']]
model_b_y = model_b_df.loc[:, '60mins_price_diff_perc']

model_b_X_train, model_b_X_test, model_b_y_train, model_b_y_test = train_test_split(model_b_X, model_b_y, test_size=0.33, random_state=42)

model_b_X_train_text = model_b_X_train.iloc[:, 0]
model_b_X_train_price_history = model_b_X_train.iloc[:, 1].apply(lambda x: split(strip()))
model_b_X_test_text = model_b_X_test.iloc[:, 0]
model_b_X_test_price_history = model_b_X_test.iloc[:, 1]
                           
model_b_corpus_list = []

for i in model_b_X_train_text:
    model_b_corpus_list.append(i.split())
    
model_b_word2vec_model = Word2Vec(model_b_corpus_list, min_count=1, size=100)
model_b_pretrained_weights = model_b_word2vec_model.wv.vectors

model_b_num_words = [len(i) for i in model_b_corpus_list]
model_b_longest_sentence_len = max(model_b_num_words)

model_b_X_train_padded = sentence_to_indices_padded(model_b_X_train_text, model_b_longest_sentence_len)
model_b_X_test_padded = sentence_to_indices_padded(model_b_X_test_text, model_b_longest_sentence_len)

model_b_X_train_input = [model_b_X_train_padded, np.array(model_b_X_train_price_history)]
model_b_X_test_input = [model_b_X_test_padded, model_b_X_test_price_history]

In [ ]:
type(np.array(model_b_X_train_input[1]))

In [ ]:
def create_model_b(pretrained_weights, longest_sentence_len, price_history_shape):
    vocab_size, embedding_size = pretrained_weights.shape
    
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False)(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='relu')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [ ]:
model_b = create_model_b(model_b_pretrained_weights, model_b_longest_sentence_len, (30,1,))

In [ ]:
model_b.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
model_b.summary()

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_b_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_b.fit(model_b_X_train_input, model_b_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

In [ ]:
model_b_X_train_input[1][0]